In [1]:
import torch
import torchvision
from torch_fast_box_ops import box_convert

In [5]:
# Create a box tensor with requires_grad=True
box_cxcywh = torch.tensor([[10.0, 20.0, 10.0, 10.0]], requires_grad=True)  # cxcywh format
box_cxcywh.register_hook(lambda grad: print("tv_cxcywh", grad))

# Convert from xyxy to cxcywh
box_xyxy = torchvision.ops.box_convert(box_cxcywh, in_fmt="cxcywh", out_fmt="xyxy")
box_xyxy.register_hook(lambda grad: print("tv_xyxy", grad))

tbo_cxcywh = box_cxcywh.clone()
tbo_xyxy = box_convert(box_cxcywh, in_fmt="cxcywh", out_fmt="xyxy")
tbo_xyxy.register_hook(lambda grad: print("tbo_xyxy", grad))

target_xyxy = torch.tensor([[4., 15., 16., 25.]])  # target in xyxy format

# Let's sum the output to get a scalar and call backward
torchvision.ops.generalized_box_iou_loss(box_xyxy, target_xyxy).sum().backward()
torchvision.ops.generalized_box_iou_loss(tbo_xyxy, target_xyxy).sum().backward()

# Show the gradients with respect to the input box
print("Input box:", box_cxcywh)
print("Converted box (xyxy):", box_xyxy)
print("Gradients:", box_cxcywh.grad)


tv_xyxy tensor([[ 0.0833, -0.0014, -0.0833,  0.0014]])
tv_cxcywh tensor([[ 0.0000,  0.0000, -0.0833,  0.0014]])
tbo_xyxy tensor([[ 0.0833, -0.0014, -0.0833,  0.0014]])
tv_cxcywh tensor([[ 0.0000,  0.0000, -0.0833,  0.0014]])
Input box: tensor([[10., 20., 10., 10.]], requires_grad=True)
Converted box (xyxy): tensor([[ 5., 15., 15., 25.]], grad_fn=<StackBackward0>)
Gradients: tensor([[ 0.0000,  0.0000, -0.1667,  0.0028]])
